In [1]:
%pip install pyarrow requests pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pyarrow.parquet as pq

# 1. SETUP: Define paths and targets
# Note: Use 'r' before the string for Windows paths
file_path = r"C:\Users\Gangadhar\OneDrive\Study\EBS\Hackathon\Business-Analytics-Hackathon\data\data-2025-12.parquet"

# Stations relevant to EBS students (Frankfurt -> Rheingau line)
target_stations = [
    "Frankfurt(Main)Hbf",
    "Frankfurt-Höchst",
    "Wiesbaden Hbf",
    "Wiesbaden-Biebrich",
    "Eltville",
    "Oestrich-Winkel",
    "Geisenheim",
    "Rüdesheim(Rhein)",
    "Mainz Hbf"  # Many students commute from Mainz too
]

print("⏳ Starting memory-efficient load...")

try:
    # 2. LOAD: Read ONLY necessary columns to save RAM
    # We leave out 'xml_station_name', 'train_line_ride_id', etc. for now
    table = pq.read_table(
        file_path, 
        columns=[
            'station_name', 
            'time', 
            'delay_in_min', 
            'is_canceled', 
            'train_type',
            'train_name', 
            'final_destination_station'
        ]
    )
    
    # Convert to Pandas DataFrame
    df = table.to_pandas()
    print(f"   Raw data loaded: {len(df):,} rows")
    
    # 3. FILTER: Keep only rows for our specific stations
    df_filtered = df[df['station_name'].isin(target_stations)].copy()
    
    # 4. CLEANING: Fix data types
    df_filtered['time'] = pd.to_datetime(df_filtered['time'])
    
    # Create Analysis Features
    df_filtered['hour'] = df_filtered['time'].dt.hour
    df_filtered['weekday'] = df_filtered['time'].dt.day_name()
    
    # Define "Delayed" (Standard is often > 5 minutes)
    df_filtered['is_delayed'] = (df_filtered['delay_in_min'] > 5).astype(int)
    
    print("✅ Processing Complete!")
    print(f"   Filtered Data (EBS Region): {len(df_filtered):,} rows")
    
    # 5. PREVIEW
    display(df_filtered.head())
    
    # 6. OPTIONAL: Save this small file for quick access later
    # This creates a lightweight file you can load instantly for analysis
    output_file = "ebs_commute_data.csv"
    df_filtered.to_csv(output_file, index=False)
    print(f"💾 Saved filtered dataset to: {output_file}")

except Exception as e:
    print(f"❌ Error: {e}")

⏳ Starting memory-efficient load...
   Raw data loaded: 15,463,467 rows
✅ Processing Complete!
   Filtered Data (EBS Region): 52,275 rows


,station_name,time,delay_in_min,is_canceled,train_type,train_name,final_destination_station,hour,weekday,is_delayed
5,Frankfurt-Höchst,2025-12-01 00:00:00,1,False,STN,STN RB12,Königstein(Taunus),0,Monday,0
561,Mainz Hbf,2025-12-01 00:02:00,0,False,S,S 8,Frankfurt Hbf (tief),0,Monday,0
1471,Eltville,2025-12-01 00:06:00,0,False,Bus,Bus RB10,Kaub,0,Monday,0
1485,Wiesbaden Hbf,2025-12-01 00:06:00,1,False,S,S 1,Rödermark-Ober Roden,0,Monday,0
1656,Mainz Hbf,2025-12-01 00:07:00,7,False,S,S 8,Wiesbaden Hbf,0,Monday,1


💾 Saved filtered dataset to: ebs_commute_data.csv
